In [1]:
import os
import sys
parent_of_parent = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(parent_of_parent)

In [2]:
from lerobot.common.robot_devices.motors.configs import DynamixelMotorsBusConfig
from lerobot.common.robot_devices.motors.dynamixel import DynamixelMotorsBus

follower_port = "/dev/ttyACM2"
leader_port = "/dev/ttyACM5"

leader_config = DynamixelMotorsBusConfig(
    port=leader_port,
    motors={
        # name: (index, model)
        "shoulder_pan": (1, "xl330-m077"),
        "shoulder_lift": (2, "xl330-m077"),
        "elbow_flex": (3, "xl330-m077"),
        "wrist_flex": (4, "xl330-m077"),
        "wrist_roll": (5, "xl330-m077"),
        "gripper": (6, "xl330-m077"),
    },
)

follower_config = DynamixelMotorsBusConfig(
    port=follower_port,
    motors={
        # name: (index, model)
        "shoulder_pan": (1, "xl430-w250"),
        "shoulder_lift": (2, "xl430-w250"),
        "elbow_flex": (3, "xl330-m288"),
        "wrist_flex": (4, "xl330-m288"),
        "wrist_roll": (5, "xl330-m288"),
        "gripper": (6, "xl330-m288"),
    },
)

leader_arm = DynamixelMotorsBus(leader_config)
follower_arm = DynamixelMotorsBus(follower_config)

In [3]:
# configure leader and follower arm
leader_arm.connect()
follower_arm.connect()

In [4]:
# now we are done configuring! test read/write
leader_pos = leader_arm.read("Present_Position")
print(leader_pos)

[1680 2363  -28 3391  980 1920]


In [6]:
follower_pos = follower_arm.read("Present_Position")
print(follower_pos)


[2348 1179 1785 1622 2131 2198]


control the follower arm

In [9]:
from lerobot.common.robot_devices.motors.dynamixel import TorqueMode

# DO NOT ATTEMPT TO MANUALLY MOVE ARM WHILE TORQUE IS ENABLED
follower_arm.write("Torque_Enable", TorqueMode.ENABLED.value)

In [10]:
# Get the current position
position = follower_arm.read("Present_Position")

# Update first motor (shoulder_pan) position by +10 steps
position[0] += 10
follower_arm.write("Goal_Position", position)

# Update all motors position by -30 steps
position -= 30
follower_arm.write("Goal_Position", position)


In [11]:
# Update gripper by +30 steps. You can see it opening wider!
position[-1] += 30
follower_arm.write("Goal_Position", position[-1], "gripper")

In [12]:
# disable torque
follower_arm.write("Torque_Enable", TorqueMode.DISABLED.value)

In [8]:
leader_arm.disconnect()
follower_arm.disconnect()

RobotDeviceNotConnectedError: DynamixelMotorsBus(/dev/ttyACM5) is not connected. Try running `motors_bus.connect()` first.